<a href="https://colab.research.google.com/github/MarcoParola/structural_health_monitoring/blob/main/testing_convolutional_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Testing Convolutional models**

In this notebook we show an evaluation of the models built and trained on "convolutional_models.ipynb". More in detail we want to observe how the models trained on some conditions (signal corruption) work in different scenarios

## Link Drive, import packages and declare global variables

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import r2_score, mean_absolute_error, confusion_matrix
import tensorflow as tf

projectPath = 'drive/MyDrive/Colab Notebooks/thesis2/'

NUMBER_OF_CLASS = 9
TRUNCATE = 9000

# the most usefull sensors are: 1,5,9,12,14,15
filter_indeces = [1,5,9,12,14,15]

earlyStopCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     mode='min', verbose=1, 
                                                     patience=20, 
                                                     restore_best_weights=True)

In [3]:
%cd $projectPath
%run utils.ipynb
%cd '../../../..'

/content/drive/.shortcut-targets-by-id/1RdMs44loEcFGBtBOenib5G7znvrr04KT/thesis2
/content


Training set and test are already divided

In [4]:
data = np.load(DATASET)

labels = load_dataset('damage_labels')
levels = load_dataset('damage_levels')

randomize = np.arange(len(labels))
np.random.shuffle(randomize)
data = data[randomize,:,:]
labels = labels[randomize]
levels = levels[randomize]

dataset = data[0:TRUNCATE, :, :]
testset = data[TRUNCATE:len(data), :, :]

labels_train = labels[0:TRUNCATE]
levels_train = levels[0:TRUNCATE]
labels_test = labels[TRUNCATE:len(labels)]
levels_test = levels[TRUNCATE:len(levels)]

labels_train = to_categorical(labels_train)
labels_test = to_categorical(labels_test)

print(dataset.shape)
print(testset.shape)

(9999,)
(9999,)
(9000, 1751, 6)
(999, 1751, 6)


# **Classification**

### **Distortion robustness testing** 


Test how a classifier trained on data without distortion solves the damage localization task using corrupted data

In [6]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_classifier.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateClassifier(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       112
           1       1.00      0.99      1.00       104
           2       1.00      1.00      1.00       121
           3       1.00      1.00      1.00       122
           4       1.00      1.00      1.00       111
           5       1.00      1.00      1.00       127
           6       1.00      1.00      1.00       101
           7       1.00      1.00      1.00        97
           8       1.00      1.00      1.00       104

    accuracy                           1.00       999
   macro avg       1.00      1.00      1.00       999
weighted avg       1.00      1.00      1.00       999

--------------------- 1 1 ---------------------
SNR: 56.623273537352745 

              precision    recall  f1-score   support

           0       0.46      1.00      0.63       112
           1       0.56      0.05      0.09       104
           2       1.00      0.92      0.96    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.21      0.54      0.30       112
           1       0.00      0.00      0.00       104
           2       0.00      0.00      0.00       121
           3       0.15      0.30      0.20       122
           4       0.50      0.03      0.05       111
           5       1.00      0.38      0.55       127
           6       0.39      0.87      0.54       101
           7       1.00      0.02      0.04        97
           8       0.56      0.83      0.67       104

    accuracy                           0.32       999
   macro avg       0.42      0.33      0.26       999
weighted avg       0.42      0.32      0.26       999

--------------------- 1 5 ---------------------
SNR: 2.2645772487521403 



...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.15      0.11      0.12       112
           1       0.00      0.00      0.00       104
           2       0.22      0.05      0.08       121
           3       0.24      0.04      0.07       122
           4       0.00      0.00      0.00       111
           5       0.18      0.14      0.16       127
           6       0.10      0.50      0.16       101
           7       0.00      0.00      0.00        97
           8       0.11      0.26      0.15       104

    accuracy                           0.12       999
   macro avg       0.11      0.12      0.08       999
weighted avg       0.12      0.12      0.09       999

--------------------- 11 10 ---------------------
SNR: 0.5664976842394184 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.08      0.05      0.07       112
           1       0.00      0.00      0.00       104
           2       0.13      0.02      0.03       121
           3       0.10      0.02      0.03       122
           4       0.00      0.00      0.00       111
           5       0.07      0.05      0.06       127
           6       0.10      0.55      0.18       101
           7       0.00      0.00      0.00        97
           8       0.11      0.28      0.15       104

    accuracy                           0.10       999
   macro avg       0.07      0.11      0.06       999
weighted avg       0.07      0.10      0.06       999

--------------------- 11 11 ---------------------
SNR: 0.46802446926826446 

              precision    recall  f1-score   support

           0       0.16      0.12      0.14       112
           1       0.00      0.00      0.00       104
           2       0.09      0.02      0.03       121
 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test how a classifier trained on noisy data solves the damage localization task using corrupted data

In [7]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_noisy_classifier.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateClassifier(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.61      0.98      0.76       112
           1       0.80      0.57      0.66       104
           2       1.00      0.90      0.95       121
           3       0.64      0.41      0.50       122
           4       1.00      0.89      0.94       111
           5       1.00      1.00      1.00       127
           6       0.71      0.96      0.82       101
           7       1.00      0.96      0.98        97
           8       1.00      1.00      1.00       104

    accuracy                           0.85       999
   macro avg       0.86      0.85      0.85       999
weighted avg       0.86      0.85      0.84       999

--------------------- 1 1 ---------------------
SNR: 56.65439445788514 

              precision    recall  f1-score   support

           0       0.59      0.96      0.73       112
           1       0.72      0.41      0.52       104
           2       1.00      0.91      0.95     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.40      0.55      0.47       112
           1       0.41      0.28      0.33       104
           2       0.89      0.62      0.73       121
           3       0.31      0.18      0.23       122
           4       0.89      0.78      0.83       111
           5       0.93      0.98      0.95       127
           6       0.48      0.81      0.61       101
           7       0.90      0.77      0.83        97
           8       0.75      0.98      0.85       104

    accuracy                           0.66       999
   macro avg       0.66      0.66      0.65       999
weighted avg       0.67      0.66      0.65       999

--------------------- 2 1 ---------------------
SNR: 56.61356224307607 

              precision    recall  f1-score   support

           0       0.37      0.51      0.43       112
           1       0.35      0.27      0.31       104
           2       0.87      0.64      0.73     

...

SNR: inf 

              precision    recall  f1-score   support

           0       0.22      0.06      0.10       112
           1       0.06      0.01      0.02       104
           2       0.14      0.12      0.13       121
           3       0.20      0.01      0.02       122
           4       0.11      0.10      0.10       111
           5       0.37      0.22      0.28       127
           6       0.16      0.35      0.22       101
           7       0.12      0.20      0.15        97
           8       0.11      0.32      0.17       104

    accuracy                           0.15       999
   macro avg       0.17      0.15      0.13       999
weighted avg       0.17      0.15      0.13       999

--------------------- 9 1 ---------------------
SNR: 56.66080725018451 

              precision    recall  f1-score   support

           0       0.05      0.01      0.02       112
           1       0.25      0.03      0.05       104
           2       0.13      0.12      0.13     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.24      0.06      0.10       112
           1       0.20      0.02      0.04       104
           2       0.12      0.10      0.11       121
           3       0.25      0.01      0.02       122
           4       0.23      0.21      0.22       111
           5       0.37      0.20      0.26       127
           6       0.13      0.30      0.18       101
           7       0.12      0.20      0.15        97
           8       0.10      0.30      0.15       104

    accuracy                           0.15       999
   macro avg       0.20      0.15      0.14       999
weighted avg       0.20      0.15      0.14       999

--------------------- 10 1 ---------------------
SNR: 56.607516966654714 

              precision    recall  f1-score   support

           0       0.14      0.03      0.04       112
           1       0.15      0.02      0.03       104
           2       0.18      0.17      0.17   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

              precision    recall  f1-score   support

           0       0.14      0.04      0.06       112
           1       0.00      0.00      0.00       104
           2       0.11      0.10      0.10       121
           3       0.00      0.00      0.00       122
           4       0.14      0.12      0.13       111
           5       0.30      0.19      0.23       127
           6       0.13      0.29      0.18       101
           7       0.14      0.20      0.16        97
           8       0.14      0.43      0.22       104

    accuracy                           0.15       999
   macro avg       0.12      0.15      0.12       999
weighted avg       0.12      0.15      0.12       999

--------------------- 11 1 ---------------------
SNR: 56.66252204704476 

              precision    recall  f1-score   support

           0       0.15      0.04      0.06       112
           1       0.00      0.00      0.00       104
           2       0.13      0.13      0.13    

# **Regression**

### **Distortion robustness testing** 


Test how a classifier trained on data without distortion solves the damage quantification task using corrupted data

In [11]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_regressor.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateRegressor(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  0.8147476446068015 
MAE 0.01557650449142394
--------------------- 1 1 ---------------------
SNR: 56.63638525647478 

R^2:  -38.927296256835994 
MAE 0.3526025311089822
--------------------- 1 2 ---------------------
SNR: 14.160435475395113 

R^2:  -38.06531605049484 
MAE 0.34812177992055315
--------------------- 1 3 ---------------------
SNR: 6.2888328760237915 

R^2:  -35.38085617601947 
MAE 0.33554540998081017
--------------------- 1 4 ---------------------
SNR: 3.536759500705271 

R^2:  -47.22211434254429 
MAE 0.3872440171692555
--------------------- 1 5 ---------------------
SNR: 2.2635830010329587 

R^2:  -64.18174453352616 
MAE 0.451302031958518
--------------------- 1 6 ---------------------
SNR: 1.5730269838700401 

R^2:  -84.86518704007798 
MAE 0.5188602735769254
--------------------- 1 7 ---------------------
SNR: 1.1548699566221918 

R^2:  -109.75749945932125 
MAE 0.5899501118177618
--------------------- 1 8 ---------------------
SNR: 0.8843224612087613 

R^2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -11.621931590318576 
MAE 0.17381319361247147
--------------------- 2 1 ---------------------
SNR: 56.59079693971379 

R^2:  -66.91241776616745 
MAE 0.45613271509536346
--------------------- 2 2 ---------------------
SNR: 14.157325611228247 

R^2:  -59.01345272046906 
MAE 0.4286851078203581
--------------------- 2 3 ---------------------
SNR: 6.287367744781969 

R^2:  -49.964796119532934 
MAE 0.39624989114697584
--------------------- 2 4 ---------------------
SNR: 3.5360937986479737 

R^2:  -59.628314994324 
MAE 0.43342134784503933
--------------------- 2 5 ---------------------
SNR: 2.2637228892113304 

R^2:  -76.54404922224418 
MAE 0.4916136297458614
--------------------- 2 6 ---------------------
SNR: 1.5728364553007994 

R^2:  -97.06260835332806 
MAE 0.5540046901158475
--------------------- 2 7 ---------------------
SNR: 1.1551475059515743 

R^2:  -121.05291854765579 
MAE 0.6189465696828115
--------------------- 2 8 ---------------------
SNR: 0.884553340700022 

R^2

...

SNR: inf 

R^2:  -140.3207315598405 
MAE 0.5625017000496609
--------------------- 9 1 ---------------------
SNR: 56.60935690042858 

R^2:  -237.5251903741203 
MAE 0.8048264350103277
--------------------- 9 2 ---------------------
SNR: 14.155188401641036 

R^2:  -215.69331144527484 
MAE 0.7686635447178249
--------------------- 9 3 ---------------------
SNR: 6.292429770400486 

R^2:  -184.25880967539848 
MAE 0.7134838832889802
--------------------- 9 4 ---------------------
SNR: 3.5398872209877728 

R^2:  -196.26071798303988 
MAE 0.7428554313041755
--------------------- 9 5 ---------------------
SNR: 2.264428179469281 

R^2:  -209.4395308818996 
MAE 0.7779889331527152
--------------------- 9 6 ---------------------
SNR: 1.573309058982125 

R^2:  -231.17149638548247 
MAE 0.8239926394957771
--------------------- 9 7 ---------------------
SNR: 1.1554378438188808 

R^2:  -263.0297113457597 
MAE 0.8849068060135789
--------------------- 9 8 ---------------------
SNR: 0.8842407161919733 

R^2: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -150.70102654486956 
MAE 0.5964692314487052
--------------------- 10 1 ---------------------
SNR: 56.60487475726584 

R^2:  -262.8654928097169 
MAE 0.8363948177250508
--------------------- 10 2 ---------------------
SNR: 14.146488885923995 

R^2:  -229.3257259530927 
MAE 0.7886760713481062
--------------------- 10 3 ---------------------
SNR: 6.288120300278675 

R^2:  -208.04892535135025 
MAE 0.7540703819938579
--------------------- 10 4 ---------------------
SNR: 3.5413842376318643 

R^2:  -220.03001723992114 
MAE 0.7775967092924261
--------------------- 10 5 ---------------------
SNR: 2.266394450822338 

R^2:  -231.2913446395832 
MAE 0.8101642307657351
--------------------- 10 6 ---------------------
SNR: 1.573700138253824 

R^2:  -255.0441881318863 
MAE 0.862269857131179
--------------------- 10 7 ---------------------
SNR: 1.1549858737420564 

R^2:  -275.90917449233314 
MAE 0.9052353063822719
--------------------- 10 8 ---------------------
SNR: 0.8857867232779995 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -157.6942655594029 
MAE 0.6103143080299652
--------------------- 11 1 ---------------------
SNR: 56.61521275917697 

R^2:  -280.47898103407726 
MAE 0.8670426413661816
--------------------- 11 2 ---------------------
SNR: 14.160573201538744 

R^2:  -253.93782222193298 
MAE 0.8217883921323209
--------------------- 11 3 ---------------------
SNR: 6.292406422901081 

R^2:  -225.83434409877927 
MAE 0.7804290470141162
--------------------- 11 4 ---------------------
SNR: 3.5412311147542743 

R^2:  -234.91367501077613 
MAE 0.8042255299785565
--------------------- 11 5 ---------------------
SNR: 2.2639918983041225 

R^2:  -240.98856339759016 
MAE 0.8279339595800138
--------------------- 11 6 ---------------------
SNR: 1.572006871172918 

R^2:  -275.97240107536055 
MAE 0.8915320517742092
--------------------- 11 7 ---------------------
SNR: 1.1555514067568726 

R^2:  -294.2176598724226 
MAE 0.9335557305141942
--------------------- 11 8 ---------------------
SNR: 0.8846336931990

Test how a classifier trained on noisy data solves the damage quantification task using corrupted data

In [12]:
model = keras.models.load_model(os.path.join(MODELS_PATH, "conv_noisy_regressor.h5"))

for delay in range(1,12):
  for noise_i in range(12):
    print('---------------------', delay, noise_i, '---------------------' )
    
    t = testset.copy()
    noise = 0.06 * noise_i * (np.random.randn(len(testset), len(testset[0]), len(testset[0,0])) *2 -1)
    t = t + noise

    power_testset = np.sum( np.abs(testset)**2 ) / (len(testset)* len(testset[0])* len(testset[0,0]))
    power_noise = np.sum( np.abs(noise)**2 ) / (len(noise)* len(noise[0])* len(noise[0,0]))
    print('SNR:', power_testset / power_noise, '\n')


    for i in range(len(t)):
      for k in range(len(t[0][0])):
        desync = np.random.randint(delay)

        tmp = t[i,:,k]
        for j in range(desync):
          tmp = np.append(tmp, [t[i,-1,k]])

        for j in range(desync):
          tmp = np.delete(tmp, 0)

        t[i,:,k] = tmp
        
    evaluateRegressor(model, t, levels_test)

--------------------- 1 0 ---------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -3.207517304791552 
MAE 0.10500809055840277
--------------------- 1 1 ---------------------
SNR: 56.6364851496526 

R^2:  -2.3438323629625604 
MAE 0.09142722536513923
--------------------- 1 2 ---------------------
SNR: 14.157643397111691 

R^2:  -1.1815246905758592 
MAE 0.06917051426487283
--------------------- 1 3 ---------------------
SNR: 6.291881369338687 

R^2:  -0.41887727655736806 
MAE 0.0522319883761184
--------------------- 1 4 ---------------------
SNR: 3.5362048849266 

R^2:  0.062309318009946324 
MAE 0.042269602245797615
--------------------- 1 5 ---------------------
SNR: 2.265257307893074 

R^2:  0.20248935003694823 
MAE 0.04094957700808175
--------------------- 1 6 ---------------------
SNR: 1.5735790679255433 

R^2:  0.12637308774308753 
MAE 0.04391632161939418
--------------------- 1 7 ---------------------
SNR: 1.155642648562956 

R^2:  -0.2606657511087689 
MAE 0.052644831814543505
--------------------- 1 8 ---------------------
SNR: 0.88428075204014

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -2.712150478474621 
MAE 0.09648854988612118
--------------------- 2 1 ---------------------
SNR: 56.65732619251253 

R^2:  -2.0221202147291333 
MAE 0.08475344817166872
--------------------- 2 2 ---------------------
SNR: 14.160507510530236 

R^2:  -1.0253968808721186 
MAE 0.0648879847083019
--------------------- 2 3 ---------------------
SNR: 6.293880104988117 

R^2:  -0.32056471387712016 
MAE 0.051281881339745955
--------------------- 2 4 ---------------------
SNR: 3.538498213088436 

R^2:  0.05166807883718516 
MAE 0.04392574417781114
--------------------- 2 5 ---------------------
SNR: 2.264830337621488 

R^2:  0.14493104020403258 
MAE 0.042751774124010426
--------------------- 2 6 ---------------------
SNR: 1.5730077658845951 

R^2:  -0.0022745942611104297 
MAE 0.04737059478538244
--------------------- 2 7 ---------------------
SNR: 1.1562615263569642 

R^2:  -0.4360750529068764 
MAE 0.05625639762788397
--------------------- 2 8 ---------------------
SNR: 0.88493867

...

SNR: inf 

R^2:  -7.124693199994429 
MAE 0.1142008529215636
--------------------- 9 1 ---------------------
SNR: 56.62599162922192 

R^2:  -5.769519893049856 
MAE 0.10871926652098395
--------------------- 9 2 ---------------------
SNR: 14.154360968254055 

R^2:  -6.662783091535009 
MAE 0.1128098200171365
--------------------- 9 3 ---------------------
SNR: 6.291198684936014 

R^2:  -6.506455831897206 
MAE 0.11155066094648804
--------------------- 9 4 ---------------------
SNR: 3.5357462783262656 

R^2:  -6.306348446339186 
MAE 0.11212955059908818
--------------------- 9 5 ---------------------
SNR: 2.263719229646384 

R^2:  -6.784536840163713 
MAE 0.11888489361064165
--------------------- 9 6 ---------------------
SNR: 1.573324486977791 

R^2:  -8.266932086638397 
MAE 0.13143589880869028
--------------------- 9 7 ---------------------
SNR: 1.1547773048939272 

R^2:  -8.0306187554485 
MAE 0.1323341924400893
--------------------- 9 8 ---------------------
SNR: 0.885107464258668 

R^2:  -

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -7.444881070596935 
MAE 0.12058015636938292
--------------------- 10 1 ---------------------
SNR: 56.63177893972687 

R^2:  -7.224901569395774 
MAE 0.11597674878932132
--------------------- 10 2 ---------------------
SNR: 14.16085538733133 

R^2:  -8.175453510190247 
MAE 0.12117751629769469
--------------------- 10 3 ---------------------
SNR: 6.295365247840707 

R^2:  -7.463735606146633 
MAE 0.11892253592480412
--------------------- 10 4 ---------------------
SNR: 3.5396406039177335 

R^2:  -8.138900394331207 
MAE 0.12502837939577133
--------------------- 10 5 ---------------------
SNR: 2.2639821396218855 

R^2:  -8.218591830599726 
MAE 0.12871412635367865
--------------------- 10 6 ---------------------
SNR: 1.5717179911662553 

R^2:  -8.401112310284065 
MAE 0.13303791214611388
--------------------- 10 7 ---------------------
SNR: 1.1557989213003192 

R^2:  -10.79142106120488 
MAE 0.15150618771042504
--------------------- 10 8 ---------------------
SNR: 0.88476270590

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


SNR: inf 

R^2:  -9.377642283348184 
MAE 0.12733930943849817
--------------------- 11 1 ---------------------
SNR: 56.59479690937643 

R^2:  -7.871062066874382 
MAE 0.12084024396977221
--------------------- 11 2 ---------------------
SNR: 14.155335834921026 

R^2:  -7.979408317301832 
MAE 0.12410177984378669
--------------------- 11 3 ---------------------
SNR: 6.294444094247714 

R^2:  -8.89349649542241 
MAE 0.12926683655800186
--------------------- 11 4 ---------------------
SNR: 3.5391269098016376 

R^2:  -9.014594325437107 
MAE 0.13233899603188026
--------------------- 11 5 ---------------------
SNR: 2.2661038528587047 

R^2:  -9.71803099134202 
MAE 0.13992305370725025
--------------------- 11 6 ---------------------
SNR: 1.573122412005497 

R^2:  -10.035053723951814 
MAE 0.14733196466684292
--------------------- 11 7 ---------------------
SNR: 1.1553113738652072 

R^2:  -11.424124557246719 
MAE 0.15749966306667498
--------------------- 11 8 ---------------------
SNR: 0.88515971954